In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pointCollection as pc

In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [3]:

from LSsurf.fd_grid import fd_grid
from LSsurf.lin_op import lin_op
import scipy.sparse as sp
from LSsurf.smooth_xytb_fit import sum_cell_area
from LSsurf.smooth_xytb_fit import calc_cell_area
from LSsurf.smooth_xytb_fit import setup_averaging_ops


In [4]:

xc=np.array([0, -5.e5])
deltas=[100., 100.]
bounds=[xc[1]+np.array([-3.e4, 3.e4]), xc[0]+np.array([-3.e4, 3.e4])]
srs_proj4='+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs '

grid_z0=fd_grid(bounds, deltas, srs_proj4=srs_proj4)
grid_dz=fd_grid(bounds+[np.array([0, 2])], [1.e3, 1.e3, 0.25])

grid_10km=fd_grid(bounds+[np.array([0, 2])], [1.e4, 1.e4, 0.25])

[xg, yg]=np.meshgrid(grid_z0.ctrs[1], grid_z0.ctrs[0])
#mask[:,grid_z0.ctrs[1]<np.mean(grid_z0.ctrs[1])-150]=1
mask=np.zeros(grid_z0.ctrs[0].size*np.array([1,1]))
mask[np.abs((xg-xc[0])+1j*(yg-xc[1]))<12000]=1

grid_z0.mask=mask


cell_area_100m = calc_cell_area(grid_z0)
cell_area_1km, op = sum_cell_area(grid_z0, grid_dz, return_op=True)
args={'avg_scales':[1.e4], 'dzdt_lags':[1, 4]}

ops=setup_averaging_ops(grid_dz, grid_dz.col_N, args, cell_area=cell_area_1km)
grid_10km=ops['avg_dz_10000m'].dst_grid

1. Check if the cell area is the same between 100 m and 10 km

In [5]:
areas_100m_1km_10km=[np.sum(cell_area_100m*mask), np.sum(cell_area_1km), np.sum(grid_10km.cell_area)]

print(areas_100m_1km_10km)
print(np.diff(np.array(areas_100m_1km_10km)))

[479113442.39076686, 479113442.3907669, 479113442.39076686]
[ 5.96046448e-08 -5.96046448e-08]


2. Do the calculation the way it's done in the ATL11_to_ATL15 code!  The difference [before I fixed it] was that setup_averaging_ops did not get a "cell_area" keyword.

In [10]:
grid_z01=fd_grid(bounds, deltas, srs_proj4=srs_proj4)
grid_z01.mask=mask
grid_dz1=fd_grid(bounds+[np.array([0, 2])], [1.e3, 1.e3, 0.25], srs_proj4=srs_proj4)

grid_z01.cell_area=calc_cell_area(grid_z01)
grid_dz1.cell_area=sum_cell_area(grid_z01, grid_dz1)
grid_dz1.mask = np.round(pc.grid.data().from_dict({'x':grid_z01.ctrs[1], 'y':grid_z01.ctrs[0], 'z':mask}).interp(grid_dz1.ctrs[1], grid_dz1.ctrs[0], gridded=True))
ops1=setup_averaging_ops(grid_dz1, grid_dz1.col_N, args)

areas1_100m_1km_10km=np.array([np.sum(grid_z01.cell_area*mask), np.sum(grid_dz1.cell_area), np.sum(ops1['avg_dz_10000m'].dst_grid.cell_area)])

print(areas1_100m_1km_10km/1.e6)
print(np.diff(areas1_100m_1km_10km))

[479.11344239 479.11344239 463.08048777]
[ 5.96046448e-08 -1.60329546e+07]


In [ ]:
grid_dz1.mask

In [ ]:
plt.figure();plt.imshow(grid_z01.cell_area)

In [ ]:
op_row = lambda this_grid, subs : this_grid.global_ind([np.array([ii]) for ii in subs])

In [ ]:
op_row(grid_10km, [3, 3, 0])

In [ ]:
av_1km=op.toCSR()
print(av_1km.shape)
temp=av_1km[21*61+30,:].toarray()
print(temp.shape)
temp=temp.reshape(mask.shape)
#plt.figure(); plt.imshow(temp)
rows=np.flatnonzero(np.any(temp, axis=0))
print(rows)
print(len(rows))
#print(rows.shape)
print(temp.sum())

In [ ]:
print((np.sum(cell_area_1km)-np.sum(cell_area_100m*mask))/np.sum(cell_area_100m*mask))
print((np.sum(ops['avg_dz_10000m'].dst_grid.cell_area)-np.sum(cell_area_1km))/np.sum(cell_area_100m))

In [ ]:
fig=plt.figure(); 
hax=fig.subplots(1,3).ravel()
hax[0].imshow(cell_area_100m*mask);
hax[0].set_title(str(np.sum(cell_area_100m*mask)/1.e4))
hax[1].imshow(cell_area_1km)
hax[1].set_title(str(np.sum(cell_area_1km)/1.e4))
hax[2].imshow(ops['avg_dz_10000m'].dst_grid.cell_area)
hax[2].set_title(str(np.sum(ops['avg_dz_10000m'].dst_grid.cell_area)/1.e4))
plt.tight_layout()

In [ ]:
# the cell areas for adjacent output cells tesselate (there's no seam between them when added)
mtx_10km=ops['avg_dz_10000m'].toCSR(row_N=np.prod(grid_dz.shape))
r1=mtx_10km[op_row(grid_10km, [3, 3, 0]),:]*ops['avg_dz_10000m'].dst_grid.cell_area[3, 3]
r2=mtx_10km[op_row(grid_10km, [3, 4, 0]),:]*ops['avg_dz_10000m'].dst_grid.cell_area[3, 4]
r12=(r1+r2).toarray().reshape(grid_dz.shape)[:,:,0]
fig=plt.figure(); 
hax=fig.subplots(1, 3)
hax[0].imshow((r1).toarray().reshape(grid_dz.shape)[:,:,0])
hax[0].set_title('cell 3,3')
hax[1].imshow((r2).toarray().reshape(grid_dz.shape)[:,:,0])
hax[1].set_title('cell 3,4')
hax[2].imshow((r1+r2).toarray().reshape(grid_dz.shape)[:,:,0])
hax[2].set_title('cell 3,3+cell 3,4')


Adding up the rows for all the output cells (multiplied by the output cell areas) gives the input cell area.

In [ ]:
rtot=np.zeros_like(mtx_10km[op_row(grid_10km, [3, 3, 0]),:])
for row in range(6):
    for col in range(6):
        rtot += mtx_10km[op_row(grid_10km, [row, col, 0]),:]*ops['avg_dz_10000m'].dst_grid.cell_area[row, col]

fig=plt.figure(); 
hax=fig.subplots(1,2)
hmap=hax[0].imshow(rtot.toarray().reshape(grid_dz.shape)[:,:,0]);
hax[0].set_title('recovered cell area')
plt.colorbar(ax=hax[0], mappable=hmap)
hmap=hax[1].imshow(rtot.toarray().reshape(grid_dz.shape)[:,:,0]-cell_area_1km);
hax[1].set_title('recovered cell area \n- grid_10km cell area')

plt.colorbar(ax=hax[1], mappable=hmap)


In [ ]:
dir(hax[0])